In [6]:
# Imports
import torch
import torch.nn as nn

import shap
import glob
import os
import pandas as pd
import numpy as np
from monai.transforms import (
    Resize,
    ScaleIntensity,
    Compose
)
import nibabel as nib
#import models
from models import PairedMedicalDataset_Full, PairedMedicalDataset_Images, SiameseNetwork_Full, SiameseNetwork_Images
import matplotlib.pyplot as plt
from monai.networks.nets import resnet

# Model evaluation

## Images only model

In [ ]:
# Instantiate the base model (e.g., ResNet or other feature extractor)
#does not work outside luna

"""
resnet_model = torch.hub.load('Warvito/MedicalNet-models', 'medicalnet_resnet10')

# Remove the final classification layer (fc) to keep only the encoder part
encoder = nn.Sequential(*list(resnet_model.children())[:-1])
"""
encoder = resnet.resnet18(spatial_dims=3, n_input_channels=1, feed_forward=False, pretrained=True, shortcut_type="A", bias_downsample=True)
model = SiameseNetwork_Images(encoder)
model.load_state_dict(torch.load("best_model.pth", map_location='cpu'))
model.eval()

OptionalImportError: from huggingface_hub import hf_hub_download (No module named 'huggingface_hub').

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

data_dir = "L:/Basic/divi/jstoker/slicer_pdac/Master Students WS 24/Martijn/data/Training/paired_scans"
clinical_data_dir = "C:/Users/P095550/OneDrive - Amsterdam UMC/Documenten/GitHub/CRLM-morph-features"
nifti_images = sorted(glob.glob(os.path.join(data_dir, "*.nii.gz")))   

# Create pairs (e.g., first and second file are paired)
image_pairs = [(nifti_images[i], nifti_images[i + 1]) for i in range(0, len(nifti_images) - 1, 2)]


pd_labels = pd.read_csv(os.path.join(clinical_data_dir, "training_labels.csv"))
all_labels = torch.tensor(pd_labels.values.tolist()) #Fill in correct path. response, PFS, and OS

Using device: cpu


In [ ]:
batch_size = 1

dataset = PairedMedicalDataset_Images(
    image_pairs, all_labels, transform=[ScaleIntensity(), Resize((64, 256, 256), mode="trilinear")]
)

# Create DataLoaders
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
labels = []
probabilities = []


# Make single predictions
for i, (image1, image2, label) in enumerate(data_loader):
    with torch.no_grad():
        # Forward pass through the model
        output = model(image1, image2)
        probs = torch.sigmoid(output)
        probabilities.append(probs)
        labels.append(label)

labels = np.vstack(labels)
probabilities = np.vstack(probabilities)

### Threshold optimization

### Saliency maps

In [ ]:
#load images
image1_path = "L:/Basic/divi/jstoker/slicer_pdac/Master Students WS 24/Martijn/data/Training/paired_tumor_scans/CAESAR001_0_tumor.nii.gz"
image2_path = "L:/Basic/divi/jstoker/slicer_pdac/Master Students WS 24/Martijn/data/Training/paired_tumor_scans/CAESAR001_1_tumor.nii.gz"

# Load the images using nibabel
image1 = nib.load(image1_path).get_fdata()
image2 = nib.load(image2_path).get_fdata()

# Apply the transformations
transform=[ScaleIntensity(), Resize((64, 256, 256), mode="trilinear")]
transform_compose = Compose(transform)
image1 = transform_compose(image1)
image2 = transform_compose(image2)

# Convert to tensors
image1 = torch.tensor(image1, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add channel and batch dimensions
image2 = torch.tensor(image2, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add channel and batch dimensions

# Make sure images have gradients enabled
img1 = image1.clone().detach().requires_grad_(True)
img2 = image2.clone().detach().requires_grad_(True)


model.eval()
output = model(img1, img2)
target_class = torch.argmax(output, dim=1).item()
score = output[0, target_class]

# Backward for saliency
model.zero_grad()
score.backward()

# Get gradients
saliency_img1 = img1.grad.abs().squeeze().cpu().numpy()
original_img1 = img1.detach().squeeze().cpu().numpy()

# Normalize both for better visualization
saliency_img1 = (saliency_img1 - saliency_img1.min()) / (saliency_img1.max() - saliency_img1.min())
original_img1 = (original_img1 - original_img1.min()) / (original_img1.max() - original_img1.min())

# Choose a slice
slice_idx = saliency_img1.shape[1] // 2  # middle slice
saliency_slice = saliency_img1[:, slice_idx, :]  # shape [H, W]
image_slice = original_img1[:, slice_idx, :]     # shape [H, W]

# Overlay saliency
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(image_slice, cmap='gray')
plt.title("Original CT Slice")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(image_slice, cmap='gray')
plt.imshow(saliency_slice, cmap='hot', alpha=0.5)  # overlay
plt.title("Overlay: CT + Saliency")
plt.axis('off')
plt.tight_layout()
plt.show()


## Full model

In [ ]:
# Instantiate the base model (e.g., ResNet or other feature extractor)

resnet_model = torch.hub.load('Warvito/MedicalNet-models', 'medicalnet_resnet10')

# Remove the final classification layer (fc) to keep only the encoder part
encoder = nn.Sequential(*list(resnet_model.children())[:-1])

model = SiameseNetwork_Full(encoder)
model.load_state_dict(torch.load("best_model.pth", map_location='cpu'))
model.eval()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

data_dir = "/scratch/bmep/mfmakaske/training_scans/"
clinical_data_dir = "/scratch/bmep/mfmakaske/"
nifti_images = sorted(glob.glob(os.path.join(data_dir, "*.nii.gz")))   

# Create pairs (e.g., first and second file are paired)
image_pairs = [(nifti_images[i], nifti_images[i + 1]) for i in range(0, len(nifti_images) - 1, 2)]

pd_metadata = pd.read_csv(os.path.join(clinical_data_dir, "training_input.csv"))
all_metadata = torch.tensor(pd_metadata.values.tolist())

pd_labels = pd.read_csv(os.path.join(clinical_data_dir, "training_labels.csv"))
all_labels = torch.tensor(pd_labels.values.tolist()) #Fill in correct path. response, PFS, and OS

In [ ]:
dataset = PairedMedicalDataset_Full(
    image_pairs, metadata=all_metadata, labels=all_labels, transform=[ScaleIntensity(), Resize((64, 256, 256), mode="trilinear")]
)

# Create DataLoaders
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)


In [ ]:
# Make single predictions
for i, (image1, image2, metadata, label) in enumerate(data_loader):
    with torch.no_grad():
        # Forward pass through the model
        output = model(image1, image2, metadata)
        probs = torch.sigmoid(output)
    print(probs)

### Trash code



In [53]:
class SiameseWrapper(nn.Module):
    def __init__(self, model):
        super(SiameseWrapper, self).__init__()
        self.model = model

    def forward(self, input):
        image1, image2, metadata = input
        return self.model(image1, image2, metadata)

In [ ]:
wrapped_model = SiameseWrapper(model).to("cuda")

# Use DeepExplainer for PyTorch models
explainer = shap.GradientExplainer(
    wrapped_model,
    data=(image1_bg, image2_bg, metadata_bg)
)

shap_values = explainer.shap_values((image1_test, image2_test, metadata_test))



KeyboardInterrupt



### Integrated gradients

In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from captum.attr import IntegratedGradients
from torchvision import models


# Instantiate the base model (e.g., ResNet or other feature extractor)

resnet_model = torch.hub.load('Warvito/MedicalNet-models', 'medicalnet_resnet10')

# Remove the final classification layer (fc) to keep only the encoder part
encoder = nn.Sequential(*list(resnet_model.children())[:-1])

model = SiameseNetwork(encoder)
model.load_state_dict(torch.load("best_model.pth", map_location='cpu'))

wrapped_model = SiameseWrapper(model)

wrapped_model.eval()


# Sample input
image_input = img1[0]
clinical_input = metadata[0]

# Use Integrated Gradients to explain the image input
ig = IntegratedGradients(wrapped_model)

# Explain the image input (we need to make sure that the image tensor requires gradients)
image_input.requires_grad = True
image_attr, _ = ig.attribute((img1[0], img2[0], metadata[0]), target=0, return_convergence_delta=True)

# Now explain the clinical input (again, ensure the tensor requires gradients)
clinical_input.requires_grad = True
clinical_attr, _ = ig.attribute(clinical_input, target=0, return_convergence_delta=True)

# Output the explanations
print("Image Attribution:", image_attr)
print("Clinical Data Attribution:", clinical_attr)

# You can visualize or process the attributions further


Using cache found in C:\Users\P095550/.cache\torch\hub\Warvito_MedicalNet-models_main


TypeError: SiameseWrapper.forward() takes 2 positional arguments but 4 were given